In [10]:
import numpy as np
import gym
from gym.spaces import Discrete


class SnakeEnv(gym.Env):
    SIZE=100
  
    def __init__(self, ladder_num, dices):
        self.ladder_num = ladder_num
        self.dices = dices
        self.ladders = dict()
        self.observation_space=Discrete(self.SIZE+1)
        self.action_space=Discrete(len(dices))

        ladders = dict(np.random.randint(1, self.SIZE, size=(self.ladder_num, 2)))
        
        for k,v in ladders.items():
            self.ladders[v] = k
            self.ladders[k] = v
            # print 'ladders info:'
            # print self.ladders
            # print 'dice ranges:'
            # print self.dices
        self.pos = 1

    def reset(self):
        self.pos = 1
        return self.pos

    def step(self, a):
        step = np.random.randint(1, self.dices[a] + 1)
        self.pos += step
        if self.pos == 100:
            return 100, 100, 1, {}
        elif self.pos > 100:
            self.pos = 200 - self.pos

        if self.pos in self.ladders:
            self.pos = self.ladders[self.pos]
        return self.pos, -1, 0, {}

    def reward(self, s):
        if s == 100:
            return 100
        else:
            return -1

    def render(self):
        pass


class TableAgent(object):
    def __init__(self, env):
        self.s_len = env.observation_space.n
        self.a_len = env.action_space.n

        self.r = [env.reward(s) for s in range(0, self.s_len)]
        self.pi = np.array([0 for s in range(0, self.s_len)])
        self.p = np.zeros([self.a_len, self.s_len, self.s_len], dtype=np.float) # p(s'|s,a)

        ladder_move = np.vectorize(lambda x: env.ladders[x] if x in env.ladders else x)

        for i, dice in enumerate(env.dices):
            prob = 1.0 / dice
            for src in range(1, 100):
                step = np.arange(dice)
                step += src
                step = np.piecewise(step, [step > 100, step <= 100],
                    [lambda x: 200 - x, lambda x: x])
                step = ladder_move(step)
                for dst in step:
                    self.p[i, src, dst] += prob
        self.p[:, 100, 100]=1
        self.value_pi = np.zeros((self.s_len))
        self.value_q = np.zeros((self.s_len, self.a_len))
        self.gamma = 0.8

    def play(self, state):
        return self.pi[state]


class ModelFreeAgent(object):
    def __init__(self, env):
        self.s_len = env.observation_space.n
        self.a_len = env.action_space.n

        self.pi = np.array([0 for s in range(0, self.s_len)])
        self.value_q = np.zeros((self.s_len, self.a_len))
        self.value_n = np.zeros((self.s_len, self.a_len))
        self.gamma = 0.8

    def play(self, state, epsilon = 0):
        if np.random.rand() < epsilon:
            return np.random.randint(self.a_len)
        else:
            return self.pi[state]


def eval_game(env, policy):
    state = env.reset()
    return_val = 0
    while True:
        if isinstance(policy, TableAgent) or isinstance(policy, ModelFreeAgent):
            act = policy.play(state)
        elif isinstance(policy, list):
            act = policy[state]
        else:
            raise Error('Illegal policy')
        state, reward, terminate, _ = env.step(act)
        # print state
        return_val += reward
        if terminate:
          break
    return return_val


env = SnakeEnv(10, [3,6])
env.reset()
while True:
    state, reward, terminate, _ = env.step(0)
    print(reward, state)
    if terminate == 1:
        break

-1 4
-1 6
-1 9
-1 12
-1 10
-1 11
-1 23
-1 26
-1 28
-1 29
-1 32
-1 34
-1 75
-1 89
-1 92
-1 93
-1 96
-1 97
-1 98
-1 99
100 100


In [26]:
import numpy as np

policy_ref = [1] * 97 + [0] * 3
policy_0 = [0] * 100
policy_1 = [1] * 100

class PolicyIteration(object):
    
    def policy_evaluation(self, agent, max_iter=-1):
        iteration = 0
        while True:
            iteration += 1
            new_value_pi = agent.value_pi.copy()
            for i in range(1, agent.s_len):
                value_sas = []
                ac = agent.pi[i]
                transition = agent.p[ac, i, :]
                value_sa = np.dot(transition, agent.r + agent.gamma * agent.value_pi)
                new_value_pi[i] = value_sa
            diff = np.sqrt(np.sum(np.power(agent.value_pi - new_value_pi, 2)))
            if diff < 1e-6:
                break
            else:
                agent.value_pi = new_value_pi
            if iteration == max_iter:
                break

    def policy_improvement(self, agent):
        new_policy = np.zeros_like(agent.pi)
        for i in range(0, agent.s_len):
            for j in range(0, agent.a_len):
                agent.value_q[i, j] = np.dot(agent.p[j, i, :], agent.r + agent.gamma * agent.value_pi)
            max_act = np.argmax(agent.value_q[i, :])
            new_policy[i] = max_act
        if np.all(np.equal(new_policy, agent.pi)):
            return False
        else:
            agent.pi = new_policy
            return True

    def policy_iteration(self, agent):
        iteration = 0
        while True:
            iteration += 1
            self.policy_evaluation(agent)
            ret = self.policy_improvement(agent)
            if not ret:
                break
        print('Iter {} rounds converge'.format(iteration))


env = SnakeEnv(0, [3, 6])
agent = TableAgent(env)
pi_algo = PolicyIteration()
pi_algo.policy_iteration(agent)

print('return_pi = {}'.format(eval_game(env, agent)))
print(agent.pi)

Iter 2 rounds converge
return_pi = 71
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0]


In [28]:
import time

from contextlib import contextmanager

@contextmanager
def timer(name):
    start = time.time()
    yield
    end = time.time()
    print('{} COST: {}'.format(name, end - start))


class PolicyIterationWithTimer(PolicyIteration):
    def policy_iteration(self, agent, max_iter = -1):
        iteration = 0
        while True:
            iteration += 1
            with timer('Timer PolicyEval'):
                self.policy_evaluation(agent, max_iter)
            with timer('Timer PolicyImprove'):
                ret = self.policy_evaluation(agent, max_iter)
            if not ret:
                break


env = SnakeEnv(10, [3,6])
agent = TableAgent(env)
pi_algo = PolicyIterationWithTimer()
pi_algo.policy_iteration(agent)
print('return_pi = {}'.format(eval_game(env, agent)))
print(agent.pi)

Timer PolicyEval COST: 0.09607505798339844
Timer PolicyImprove COST: 0.0018949508666992188
return_pi = 10
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [30]:
class ValueIteration(object):
    def value_iteration(self, agent, max_iter = -1):
        iteration = 0
        while True:
            iteration += 1
            new_value_pi = np.zeros_like(agent.value_pi)
            for i in range(1, agent.s_len): # for each state
                value_sas = []
                for j in range(0, agent.a_len): # for each act
                    value_sa = np.dot(agent.p[j, i, :], agent.r + agent.gamma * agent.value_pi)
                    value_sas.append(value_sa)
                new_value_pi[i] = max(value_sas)
            diff = np.sqrt(np.sum(np.power(agent.value_pi - new_value_pi, 2)))

            if diff < 1e-6:
                break
            else:
                agent.value_pi = new_value_pi
            if iteration == max_iter:
                break
        print('Iter {} rounds converge'.format(iteration))
        for i in range(1, agent.s_len):
            for j in range(0, agent.a_len):
                agent.value_q[i, j] = np.dot(agent.p[j, i, :], agent.r + agent.gamma * agent.value_pi)
            max_act = np.argmax(agent.value_q[i, :])
            agent.pi[i] = max_act


np.random.seed(0)
env = SnakeEnv(10, [3,6])
agent = TableAgent(env)
vi_algo = ValueIteration()
vi_algo.value_iteration(agent)
print('return_pi={}'.format(eval_game(env,agent)))
print(agent.pi)

Iter 94 rounds converge
return_pi=92
[0 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0]


In [34]:
import numpy as np

# general iteration
class GeneralizedPolicyIteration(object):
    def __init__(self):
        self.pi_algo = PolicyIterationWithTimer()
        self.vi_algo = ValueIteration()

    def generalized_policy_iteration(self, agent):
       self.vi_algo.value_iteration(agent, 10)
       self.pi_algo.policy_iteration(agent, 1)

def policy_iteration_demo():
    np.random.seed(0)
    env = SnakeEnv(10, [3,6])
    agent = TableAgent(env)
    pi_algo = PolicyIterationWithTimer()
    with timer('Timer PolicyIter'):
        pi_algo.policy_iteration(agent)
    print('return_pi={}'.format(eval_game(env,agent)))

def value_iteration_demo():
    np.random.seed(0)
    env = SnakeEnv(10, [3,6])
    agent = TableAgent(env)
    pi_algo = ValueIteration()
    with timer('Timer ValueIter'):
        pi_algo.value_iteration(agent)
    print('return_pi={}'.format(eval_game(env,agent)))

def generalized_iteration_demo():
    np.random.seed(0)
    env = SnakeEnv(10, [3,6])
    agent = TableAgent(env)
    pi_algo = GeneralizedPolicyIteration()
    with timer('Timer GeneralizedIter'):
        pi_algo.generalized_policy_iteration(agent)
    print('return_pi={}'.format(eval_game(env,agent)))

if __name__ == '__main__':
    policy_iteration_demo()
    print('===========================')
    value_iteration_demo()
    print('===========================')
    generalized_iteration_demo()

Timer PolicyEval COST: 0.09385323524475098
Timer PolicyImprove COST: 0.0014247894287109375
Timer PolicyIter COST: 0.0954427719116211
return_pi=88
Iter 94 rounds converge
Timer ValueIter COST: 0.1848917007446289
return_pi=92
Iter 10 rounds converge
Timer PolicyEval COST: 0.0010371208190917969
Timer PolicyImprove COST: 0.0012159347534179688
Timer GeneralizedIter COST: 0.027844667434692383
return_pi=92
